In [107]:
import csv

my_commands = {
    "#newEntry" : {"module": "mySalary", "method": "new_entry", "notes":"insert type(incomings or outgoings), month(e.g: Nov), name, key-words*, notes*, value"},
    "#monthRecap" : {"module": "mySalary", "method": "monthRecap", "notes":"insert months(list or one month)"},
    "#monthOutgoings" : {"module": "mySalary", "method": "monthOutgoings", "notes":"insert months(list or one month)"},
    "#monthIncomings" : {"module": "mySalary", "method": "monthIncomings", "notes":"insert months(list or one month)"},
    "#getKeyEntries" : {"module": "mySalary", "method": "keyEntries", "notes":"insert: outgoings/incomings, months and the keys"},
    "#backupData" : {"module": "mySalary", "method": "backup", "notes":""}
}

In [99]:
def new_entry(type_entry, month, name,keys,notes,value):
    month = initMonth(month)
        
    for i in range(0,len(month)):
        if month[i] not in movements[type_entry]:
            movements[type_entry][month[i]] = []
        movements[type_entry][month[i]].append({"name":name, "keys": keys, "notes": notes, "value": value})

def sumOf(category,months):
    months = initMonth(months)
    
    total = 0
    for m in movements[category]:
        if m in months:
            for entry in movements[category][m]:
                total += entry["value"]
    return total

def keyEntries(category,months,keys):
    months = initMonth(months)
    
    entries = []
    for m in movements[category]:
        if m in months:
            for entry in movements[category][m]:
                flag = True
                if keys != "*":
                    keys = initKeys(keys)
                    for k in keys:
                        flag = flag and ( k in entry["keys"])
                if flag:
                    entries.append(entry)
    return entries
                

def monthRecap(months):
    return monthIncoming(months) - monthOutgoings(months)

def monthOutgoings(months):
    return sumOf("outgoings",months)

def monthIncoming(months):
    return sumOf("incomings",months)
    
def backup():
    createCsv()

def initMonth(month):
    if month == "*":
        month = ["Jan","Feb","March","April","May","June","Aug","Sep","Oct","Nov","Dec"]
    
    if not isinstance(month, list):
        month = [month]
    
    return month

def initKeys(keys):
    if keys == "*":
        list_k = []
        for c in movements:
            for m in movements[c]:
                for entry in movements[c][m]:
                    for k in entry["keys"]:
                        if k not in list_k:
                            list_k.append(k)
        keys = list_k
        
    
    if not isinstance(keys, list):
        keys = [keys]
    
    return keys
    
def createCsv(*args):
    mylist = []
    for type_m in movements:
        for date in movements[type_m]:
            for entry in movements[type_m][date]:
                myEntry = entry
                myEntry["type"] = str(type_m) 
                myEntry["date"] = str(date)
                mylist.append(myEntry)
            
    with open("movements.csv", 'wb') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        
        #header
        for elem in mylist:
            a_row = []
            for key in elem:
                a_row.append(key)                
            wr.writerow(a_row)
            break
        
        #all values
        for elem in mylist:
            a_row = []
            for key in elem:
                str_elem = ""
                if isinstance(elem[key], list):
                    separator = ","
                    for i in range(0, len(elem[key])):
                        if (i == len(elem[key]) - 1): 
                            separator = ""
                        str_elem = str_elem + elem[key][i] + separator
                else:
                    str_elem = elem[key]
                    
                a_row.append(str_elem)    
            wr.writerow(a_row)
                

In [126]:
import numpy as np
import matplotlib.pyplot as plt

def percKeyByMonth(keys, months, category):
    keys = initKeys(keys)
    months = initMonth(months)
    
    sumAll = 0
    sumKey = 0
    for monthKey in movements[category]:
        if monthKey in months:
            for entry in movements[category][monthKey]:
                flag= True
                for k in keys:
                    flag = flag and (k in entry['keys'])
                if flag:
                    sumKey = sumKey + entry["value"]
                sumAll = sumAll + entry["value"]
    return float(sumKey) / sumAll
    

def createGBars(keys, months, block, category): 
    
    keys = initKeys(keys)
    months = initMonth(months)
      
    N = len(keys)
    keyVal = ()
    keyH = ()
    restVal = ()
    for i in range(0,len(months),block):
        monthBlk = months[i:i+block]
        perc = percKeyByMonth(keys, monthBlk, category)
        
        list(keyVal).insert(i+block / block, perc*100)
        keyVal = tuple(keyVal)
        
        list(restVal).insert(i+block / block, 100 - (perc*100))
        restVal = tuple(restVal)
        
            
    #menStd = (2, 3, 4, 1, 2)
    #womenStd = (3, 5, 2, 3, 3)
    ind = np.arange(N)    # the x locations for the groups
    width = 0.35       # the width of the bars: can also be len(x) sequence

    p1 = plt.bar(ind, keyVal, width)
    p2 = plt.bar(ind, restVal, width)

    #plt.ylabel('Scores')
    #plt.title('Scores by group and gender')
    #plt.legend((p1[0], p2[0]), ('Men', 'Women'))
    plt.xticks(ind)
    plt.yticks(np.arange(0, 81, 10))

    plt.show()

In [100]:
mySalary = 1938
movements = {"outgoings":{},"incomings":{}}

new_entry("incomings","*","Salario lavoro", ["personale","lavoro","salario"], "", 1938)
new_entry("outgoings", "Nov","Giacca carrhart", ["personale","abbigliamento"], "", 240)
new_entry("outgoings","Oct","Cena compleanno", ["regalo"], "", 250)
new_entry("incomings","Oct","Regalo compleanno", ["personale","regalo"], "", 100)
new_entry("outgoings","Nov","Profumo", ["personale","cura-persona"], "", 70)
new_entry("outgoings","Nov","HD SSD", ["personale","elettronica"], "", 140)
new_entry("outgoings", "Dec","Biglietto Alitalia (+Penale)", ["personale","viaggio"], "", 650)
new_entry("outgoings", "Dec","Giacca element Adam", ["regalo","abbigliamento"], "", 130)
new_entry("outgoings", "Dec","Regali per cugini (+ Quadro Jihan)", ["regalo","varie"], "", 100)
new_entry("outgoings", "Dec","Regalo per la Mamma", ["regalo","elettronica"], "", 170)
new_entry("outgoings", "Dec","Biglietti concerti", ["personale","svago"], "", 50)
new_entry("outgoings", "*","Abb. Spotify", ["personale","svago"], "", 15)
new_entry("outgoings", "Dec","Camera raspberry", ["personale","elettronica"], "", 30)
new_entry("outgoings", "Dec","Viaggio per Trieste", ["personale","viaggio"], "", 30)

In [127]:
percKeyByMonth(["personale","abbigliamento"],"*","outgoings")
createGBars(["personale","abbigliamento"], "*", 2, "outgoings")

ValueError: incompatible sizes: argument 'height' must be length 2 or scalar

In [101]:
print monthRecap("Dec")

763


In [106]:
keyEntries("outgoings","Dec",["elettronica","personale"])

[{'keys': ['personale', 'elettronica'],
  'name': 'Camera raspberry',
  'notes': '',
  'value': 30}]

In [80]:
createCsv()